In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "logs/logs/qwen-7b-lora-aligned-cpo"

# model_name = "Qwen/Qwen2.5-3B-Instruct"

llm_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/lesong/anaconda3/envs/trlenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.63it/s]


In [4]:

def get_response(prompt, llm_model, tokenizer):
    messages = [
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(llm_model.device)

    generated_ids = llm_model.generate(
        **model_inputs,
        max_new_tokens=256
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

import jsonlines
import random


with jsonlines.open("data/cpo_data.json") as reader:
    print(max([len(obj["rejected"][0]["content"])+len(obj["rejected"][1]["content"]) for obj in reader]))
    # for obj in reader:
        # if random.random() <= 0.05:
        #     prompt = obj["chosen"][0]["content"]
        #     resp = get_response(prompt, llm_model, tokenizer)
        #     print(resp, obj["chosen"][1]["content"])

3307
